**RANDOM FOREST**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [ ]:
data= pd.read_csv('data_ml.csv')

## Preprocessing

In [ ]:
# Ego variables preprocessing

# Imputation for NAs

# mean imputation
data_merged['age'].fillna(data_merged['age'].mean(), inplace=True)

# imputation with mode
for col in ['pressure_f_num', 'pressure_p_num', 'happiness_num']:
    data_merged[col].fillna(data_merged[col].mode()[0], inplace=True)

#print(data_merged.isna().sum())

# Remove I don't knows

data = data_merged[data_merged['childwish_num'] != 3]
print(data[data['childwish_num'] == 3])

#How long have you and your partner been in a relationship? (relationship_duration)

mapping = {
    'Less than 1 year': 0.5,
    'Between 1 and 2 years': 1.5,
    'Between 2 and 3 years': 2.5,
    'Between 3 and 4 years': 3.5,
    'Between 4 and 5 years': 4.5,
    'More than 5 years': 6,
    'NA': 0
}

data['relationship_duration_num'] = data['relationship_duration'].map(mapping)
data['relationship_duration_num'] = data['relationship_duration_num'].fillna(0)

# Do you live together with your partner? (cohabiting)

def preprocess_coh(value):
    if value == 'Yes':
        return 1
    else:
        return 0

data['cohabiting_num'] = data['cohabiting'].apply(preprocess_coh)

#What kind of a cohabitation form do you have with your partner? (cohabitation_form)


def preprocess_coh_form(value):
    if value == 'Marriage':
        return 1
    elif value == 'Registered partnership':
        return 2
    elif value == 'Cohabitation contract':
        return 3
    elif value == 'No formal cohabitation form':
        return 4
    else:
        return 0

data['cohabitation_form_num'] = data['cohabitation_form'].apply(preprocess_coh_form)

# Civil status

def preprocess_status(value):
    if value == 'Married':
        return 1
    elif value == 'Divorced':
        return 2
    elif value == 'Separated':
        return 3
    elif value == 'Never been married':
        return 4
    else:
        return 0

data['civil_status_num'] = data['civil_status'].apply(preprocess_status)

# Urban

def preprocess_urban(value):
    if value == 'Extremely urban' or value == 'Very urban':
        return 1
    elif value == 'Moderately urban' or value == 'Slightly urban':
        return 2
    elif value == 'Not urban':
        return 3
    else:
        return 0

data['urban_num'] = data['urban'].apply(preprocess_urban)


# Type_dwelling

def preprocess_dwelling(value):
    if value == 'Self-owned dwelling':
        return 1
    elif value == 'Cost-free dwelling':
        return 2
    elif value == 'Rental dwelling':
        return 3
    else:
        return 0

data['type_dwelling_num'] = data['type_dwelling'].apply(preprocess_dwelling)


# Origins

def preprocess_origin(value):
    if value == 'Dutch background':
        return 1
    elif value == 'First generation foreign, Western background':
        return 2
    elif value == 'First generation foreign, non-western background':
        return 2
    elif value == 'Second generation foreign, Western background':
        return 3
    elif value == 'Second generation foreign, non-western background':
        return 4
    else:
        return 0

data['origin_num'] = data['origin'].apply(preprocess_origin)




# Preprocessing for structural variables

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Checking NAs
columns =['mean_closeness',	'mean_f2f',	'mean_nonf2f',	'mean_closeness_kin',	'mean_closeness_friends',	'mean_closeness_has_kid',	'mean_closeness_want_kid',	'mean_closeness_wants_no_kid',
             'mean_closeness_help',	'mean_closeness_talk',	'mean_f2f_kin',	'mean_f2f_friends',	'mean_f2f_has_kid',	'mean_f2f_want_kid',	'mean_f2f_wants_no_kid',	'mean_f2f_help',	'mean_f2f_talk',	'mean_nonf2f_kin',
             'mean_nonf2f_friends',	'mean_nonf2f_has_kid',	'mean_nonf2f_want_kid',	'mean_nonf2f_wants_no_kid',	'mean_nonf2f_help',	'mean_nonf2f_talk',	'mean_closeness_kin_cor',	'mean_closeness_friends_cor',
             'mean_closeness_has_kid_cor',	'mean_closeness_want_kid_cor',	'mean_closeness_wants_no_kid_cor',	'mean_closeness_help_cor',	'mean_closeness_talk_cor',	'mean_f2f_kin_cor',	'mean_f2f_friends_cor',	'mean_f2f_has_kid_cor',
             'mean_f2f_want_kid_cor',	'mean_f2f_wants_no_kid_cor',	'mean_f2f_help_cor',	'mean_f2f_talk_cor',	'mean_nonf2f_kin_cor',	'mean_nonf2f_friends_cor',
             'mean_nonf2f_has_kid_cor',	'mean_nonf2f_want_kid_cor',	'mean_nonf2f_wants_no_kid_cor',	'mean_nonf2f_help_cor',	'mean_nonf2f_talk_cor',	'no_women',	'no_older',	'no_friends',	'no_kin',	'no_high_edu',	'no_has_child',
             'no_child_total',	'no_child_u5',	'no_child_less_happy',	'no_wants_child',	'no_wants_no_child',	'no_help',	'no_talk',	'density',	'density_kin',	'density_friends',	'density_children',	'density_wantschildren',	'density_childfree',
             'density_talk',	'density_help',	'comm_1or2',	'comm_3orhigher',	'modularity',	'comp_largest',	'diameter',	'between_centr',	'degree_centr',	'avg_betweenness',	'avg_closeness',	'avg_eigenv',	'cliques',	'components',	'density_kin_cor',
             'density_friends_cor',	'density_children_cor',	'density_wantschildren_cor',	'density_childfree_cor',	'density_talk_cor',	'density_help_cor' ]

#print(data_merged[columns].isna().sum())

# create variants of vars that have 1 (lowest tie strength) if group is missing
# This replaces all NaN to 1 for selected variables and creates new variables
# NaN means e.g., that respondents had no people in network with children, so
# mean_closeness_has_kid is NaN. Imputing 1 implies the lowest possible score

for column in columns:
    if column in data_merged.columns:
        data[column].fillna(1, inplace=True)


Empty DataFrame
Columns: [nomem_encr, year_month_survey, partner, sex_partner, birthyear_partner, relationship_duration, cohabiting, cohabitation_form, has_children, num_children, num_children_biol, has_children_partner, childwish, ideal_family_size, strength_preference, when_childwish, discussed_childwish, childwish_partner, family_size_nl, happiness_children, pressure_friends, pressure_parents, happiness_children_network, names_network, questions_difficult_survey, questions_clear_survey, questions_thinking_survey, questions_interesting_survey, questions_enjoy_survey, start_date_survey, start_time_survey, end_date_survey, end_time_survey, duration, alters_reported, ties_missing, missing_alter_attr, notes, nohouse_encr, wave, sex, position_hh, birthyear, age, age_cat, age_head_hh, number_hh_members, children_at_home, lives_with_partner, civil_status, living_situation, type_dwelling, urban, primary_occ, gross_income, gross_income_imp, net_income, net_income2, net_income_imp, gross_incom

<ipython-input-3-73a28357764d>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['relationship_duration_num'] = data['relationship_duration'].map(mapping)
<ipython-input-3-73a28357764d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['relationship_duration_num'] = data['relationship_duration_num'].fillna(0)
<ipython-input-3-73a28357764d>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

# 1. Ego Model

In [ ]:
X = data[['age', 'partner_num', 'has_child_num', 'educ_bin', 'child_num', 'relationship_duration_num', 'cohabiting_num',
          'cohabitation_form_num', 'happiness_num', 'pressure_f_num', 'pressure_p_num', 'civil_status_num', 'urban_num',
          'type_dwelling_num', 'origin_num']]
y = data['childwish_num']

# Hyperparameter tuning
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False]
}

# GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train and evaluate the model with CV
grid_search.fit(X, y)


best_model = grid_search.best_estimator_

print(f"Best cross-validation accuracy: {grid_search.best_score_}")

# Cross-validation predictions
y_pred = cross_val_predict(best_model, X, y, cv=5)


print(classification_report(y, y_pred))
print("Best parameters found:", grid_search.best_params_)



Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best cross-validation accuracy: 0.5364600550964187
              precision    recall  f1-score   support

           1       0.51      0.56      0.53        89
           2       0.43      0.30      0.35        98
           4       0.55      0.67      0.60       227
           5       0.58      0.49      0.53       190

    accuracy                           0.54       604
   macro avg       0.52      0.50      0.50       604
weighted avg       0.53      0.54      0.53       604

Best parameters found: {'classifier__bootstrap': False, 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}


In [ ]:
# Calculate feature importance using permutation importance method

result = permutation_importance(best_model, X, y, n_repeats=10, random_state=42, n_jobs=-1)

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': result.importances_mean,
    'Std Dev': result.importances_std
}).sort_values(by='Importance', ascending=False)

print("\nPermutation Importance Scores:")
print(importance_df)


Permutation Importance Scores:
                      Feature  Importance   Std Dev
0                         age    0.220199  0.011566
10             pressure_p_num    0.107285  0.013507
9              pressure_f_num    0.080960  0.007921
5   relationship_duration_num    0.074834  0.002751
4                   child_num    0.054636  0.005923
7       cohabitation_form_num    0.051325  0.007587
14                 origin_num    0.044371  0.007968
3                    educ_bin    0.032616  0.006241
11           civil_status_num    0.029139  0.006335
12                  urban_num    0.029139  0.005979
13          type_dwelling_num    0.025000  0.005207
6              cohabiting_num    0.020861  0.005933
8               happiness_num    0.016556  0.005735
1                 partner_num    0.015066  0.004018
2               has_child_num    0.013576  0.004492


# 2. Network variables model

In [ ]:
X_str = data[['mean_closeness',	'mean_f2f',	'mean_nonf2f',	'mean_closeness_kin',	'mean_closeness_friends',	'mean_closeness_has_kid',	'mean_closeness_want_kid',	'mean_closeness_wants_no_kid',
             'mean_closeness_help',	'mean_closeness_talk',	'mean_f2f_kin',	'mean_f2f_friends',	'mean_f2f_has_kid',	'mean_f2f_want_kid',	'mean_f2f_wants_no_kid',	'mean_f2f_help',	'mean_f2f_talk',	'mean_nonf2f_kin',
             'mean_nonf2f_friends',	'mean_nonf2f_has_kid',	'mean_nonf2f_want_kid',	'mean_nonf2f_wants_no_kid',	'mean_nonf2f_help',	'mean_nonf2f_talk',	'mean_closeness_kin_cor',	'mean_closeness_friends_cor',
             'mean_closeness_has_kid_cor',	'mean_closeness_want_kid_cor',	'mean_closeness_wants_no_kid_cor',	'mean_closeness_help_cor',	'mean_closeness_talk_cor',	'mean_f2f_kin_cor',	'mean_f2f_friends_cor',	'mean_f2f_has_kid_cor',
             'mean_f2f_want_kid_cor',	'mean_f2f_wants_no_kid_cor',	'mean_f2f_help_cor',	'mean_f2f_talk_cor',	'mean_nonf2f_kin_cor',	'mean_nonf2f_friends_cor',
             'mean_nonf2f_has_kid_cor',	'mean_nonf2f_want_kid_cor',	'mean_nonf2f_wants_no_kid_cor',	'mean_nonf2f_help_cor',	'mean_nonf2f_talk_cor',	'no_women',	'no_older',	'no_friends',	'no_kin',	'no_high_edu',	'no_has_child',
             'no_child_total',	'no_child_u5',	'no_child_less_happy',	'no_wants_child',	'no_wants_no_child',	'no_help',	'no_talk',	'density',	'density_kin',	'density_friends',	'density_children',	'density_wantschildren',	'density_childfree',
             'density_talk',	'density_help',	'comm_1or2',	'comm_3orhigher',	'modularity',	'comp_largest',	'diameter',	'between_centr',	'degree_centr',	'avg_betweenness',	'avg_closeness',	'avg_eigenv',	'cliques',	'components',	'density_kin_cor',
             'density_friends_cor',	'density_children_cor',	'density_wantschildren_cor',	'density_childfree_cor',	'density_talk_cor',	'density_help_cor' ]]
y = data['childwish_num']

# Hyperparameter tuning
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False]
}

# GridSearchCV
grid_search_str = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train and evaluate the model with CV
grid_search_str.fit(X_str, y)


best_model_str = grid_search_str.best_estimator_

print(f"Best cross-validation accuracy: {grid_search_str.best_score_}")

# Cross-validation predictions
y_pred_str = cross_val_predict(best_model_str, X_str, y, cv=5)


print(classification_report(y, y_pred_str))
print("Best parameters found:", grid_search_str.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best cross-validation accuracy: 0.45196969696969697
              precision    recall  f1-score   support

           1       0.49      0.42      0.45        89
           2       0.41      0.22      0.29        98
           4       0.46      0.64      0.54       227
           5       0.43      0.36      0.39       190

    accuracy                           0.45       604
   macro avg       0.45      0.41      0.42       604
weighted avg       0.45      0.45      0.44       604

Best parameters found: {'classifier__bootstrap': False, 'classifier__max_depth': None, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}


# 3. Full Model

In [ ]:
X_full = data[['age', 'partner_num', 'has_child_num', 'educ_bin', 'child_num', 'relationship_duration_num','cohabiting_num',
 'cohabitation_form_num', 'happiness_num', 'pressure_f_num', 'pressure_p_num', 'civil_status_num', 'urban_num', 'type_dwelling_num','origin_num','mean_closeness',	'mean_f2f',	'mean_nonf2f',	'mean_closeness_kin',	'mean_closeness_friends',	'mean_closeness_has_kid',	'mean_closeness_want_kid',	'mean_closeness_wants_no_kid',
             'mean_closeness_help',	'mean_closeness_talk',	'mean_f2f_kin',	'mean_f2f_friends',	'mean_f2f_has_kid',	'mean_f2f_want_kid',	'mean_f2f_wants_no_kid',	'mean_f2f_help',	'mean_f2f_talk',	'mean_nonf2f_kin',
             'mean_nonf2f_friends',	'mean_nonf2f_has_kid',	'mean_nonf2f_want_kid',	'mean_nonf2f_wants_no_kid',	'mean_nonf2f_help',	'mean_nonf2f_talk',	'mean_closeness_kin_cor',	'mean_closeness_friends_cor',
             'mean_closeness_has_kid_cor',	'mean_closeness_want_kid_cor',	'mean_closeness_wants_no_kid_cor',	'mean_closeness_help_cor',	'mean_closeness_talk_cor',	'mean_f2f_kin_cor',	'mean_f2f_friends_cor',	'mean_f2f_has_kid_cor',
             'mean_f2f_want_kid_cor',	'mean_f2f_wants_no_kid_cor',	'mean_f2f_help_cor',	'mean_f2f_talk_cor',	'mean_nonf2f_kin_cor',	'mean_nonf2f_friends_cor',
             'mean_nonf2f_has_kid_cor',	'mean_nonf2f_want_kid_cor',	'mean_nonf2f_wants_no_kid_cor',	'mean_nonf2f_help_cor',	'mean_nonf2f_talk_cor',	'no_women',	'no_older',	'no_friends',	'no_kin',	'no_high_edu',	'no_has_child',
             'no_child_total',	'no_child_u5',	'no_child_less_happy',	'no_wants_child',	'no_wants_no_child',	'no_help',	'no_talk',	'density',	'density_kin',	'density_friends',	'density_children',	'density_wantschildren',	'density_childfree',
             'density_talk',	'density_help',	'comm_1or2',	'comm_3orhigher',	'modularity',	'comp_largest',	'diameter',	'between_centr',	'degree_centr',	'avg_betweenness',	'avg_closeness',	'avg_eigenv',	'cliques',	'components',	'density_kin_cor',
             'density_friends_cor',	'density_children_cor',	'density_wantschildren_cor',	'density_childfree_cor',	'density_talk_cor',	'density_help_cor' ]]
y = data['childwish_num']

# Creating a pipeline and Hyperparameter grid for tuning

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False]
}

# GridSearchCV
grid_search_full = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train and evaluate the model with CV
grid_search_full.fit(X_full, y)


best_model_full = grid_search_full.best_estimator_

print(f"Best cross-validation accuracy: {grid_search_full.best_score_}")

# Cross-validation predictions
y_pred_full = cross_val_predict(best_model_full, X_full, y, cv=5)

# Print classification report
print(classification_report(y, y_pred_full))

# Print the best parameters
print("Best parameters found:", grid_search_full.best_params_)

# Calculate feature importance using permutation importance method

result = permutation_importance(best_model_full, X_full, y, n_repeats=10, random_state=42, n_jobs=-1)

importance_df = pd.DataFrame({
    'Feature': X_full.columns,
    'Importance': result.importances_mean,
    'Std Dev': result.importances_std
}).sort_values(by='Importance', ascending=False)

print("\nPermutation Importance Scores:")
print(importance_df)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best cross-validation accuracy: 0.5066115702479339
              precision    recall  f1-score   support

           1       0.52      0.49      0.51        89
           2       0.40      0.30      0.34        98
           4       0.51      0.66      0.58       227
           5       0.54      0.44      0.49       190

    accuracy                           0.51       604
   macro avg       0.49      0.47      0.48       604
weighted avg       0.50      0.51      0.50       604

Best parameters found: {'classifier__bootstrap': False, 'classifier__max_depth': None, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 200}

Permutation Importance Scores:
                            Feature  Importance   Std Dev
0                               age    0.018709  0.003477
10                   pressure_p_num    0.010099  0.001880
9                    pressure_f_num    0.007616  0.0